In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random
# import setuptools.dist
import undetected_chromedriver as uc
import warnings
warnings.filterwarnings('ignore')

In [2]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(1)
    c.close()
    db.close()
    return True


def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists PlansetePigu
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists PlansetePigu
    (
        nuoroda text,
        firstInfo text
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass

def writeMany(lst, table, dbname) -> bool:
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''insert into {table} values ({n})'''
    q = ['?' for i in range(0,len(lst))]
    q_ = ', '.join(q)
    r_ = sql.format(table=table, n=q_)
    # print('::PROVIDED::', lst, len(lst), '::SQL COMMAND::', r_, len(q), sep='\n')
    c.execute(r_, lst)
    db.commit()
    c.close()
    db.close()
    return True


In [3]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
url = 'https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets?page=1'
# url = 'https://pigu.lt/lt/saldytuvai/'
driver.get(url)
time.sleep(5)
source = driver.page_source


driver.close()
print('Baigta!!!')

Baigta!!!


In [4]:
bs = BeautifulSoup(source, 'html.parser')
p_nuorodos = []

linkai = bs.find_all('div', {'class':'c-product-card__title'})
for linkas in linkai:
    link = linkas.find('a')['href']
    # print(link)
    p_nuorodos.append(link)

print(len(p_nuorodos))
print(p_nuorodos)


max_page_number = bs.find('a', {'class':'pageSlot-7'}).text
print(max_page_number)



59
['https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-samsung-galaxy-tab-a9-wifi-464gb-graphite?id=78014478', 'https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-xiaomi-redmi-pad-se-4128gb-graphite-gray?id=75318247', 'https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-xiaomi-redmi-pad-se-4128gb?id=75318242', 'https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-xiaomi-redmi-pad-se-11-8256gb-grey?id=78335458', 'https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-xiaomi-redmi-pad-pro-121-6gb128gb-graphite?id=86062345', 'https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-xiaomi-pad-pro-8256gb-graphite-grey?id=86298855', 'https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-samsung-galaxy-tab-a9-wifi-8128gb-graphite?id=78014483', 'https://pigu.l

In [13]:
for nuoroda in p_nuorodos[1:2]:
    print(nuoroda)
    opcijos = Options()
    opcijos.add_argument('--incognito')
    opcijos.add_argument('--ignore-certificate-errors')
    opcijos.add_argument('--ignore-ssl-errors')
    driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
    # url = nuoroda
    url = 'https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-oukitel-rt3-4g-464gb-black?id=67919111'
    driver.get(url)
    time.sleep(5)
    source = driver.page_source


    driver.quit()

https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-xiaomi-redmi-pad-se-4128gb-graphite-gray?id=75318247


In [14]:
bs = BeautifulSoup(source, 'html.parser')
firstInfo = bs.find('div', {'class':'c-product__info'}).find('h1', {'class':'c-product__name'}).text.strip()
print(firstInfo)

secondInfo = bs.find('div', {'class':'c-accordion-item__title'}).text.replace('Prekės aprašymas:', '').strip()
print(secondInfo)

# x = eval(price['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
# dict(x)
# x['meta']['sell_price']

kaina = bs.find('div', {'class':'c-product__media'})
price = eval(kaina['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
kaina2 = price['dataLayerItem'].get('price', 'nenurodyta')
print(kaina2)

# if 'price' in price['dataLayerItem']:
#     kaina2 = 'nera'
# else:
#     kaina2 = price['dataLayerItem']['price']
# print(kaina2)


# kaina = bs.find('div', {'class':'c-price h-price--xx-large h-price--new'}).text.strip()
# print(kaina)

specifikacija_keys = []
specifikacija_values = []
specifikacija = bs.find('div', {'class':'h-row h-pl--8 h-pr--8'}).find_all('tr')
for trow in specifikacija:
    td = trow.find_all('td')
    key = td[0].text.strip()
    value = td[1].text.strip()
    specifikacija_keys.append(key)
    specifikacija_values.append(value)

print(len(specifikacija_keys), specifikacija_keys)
print(len(specifikacija_values), specifikacija_values)




Planšetinis kompiuteris Oukitel RT3 4G 4/64GB Black
Oukitel RT3 4G 4/64GB Black
171
36 ['Prekės ID:', 'Kategorija:', 'Gamintojo kodas:', 'Prekės pakuočių kiekis:', 'Pakuotės išmatavimai ir svoris (1):', 'Prekės ženklas:', 'Ekrano tipas:', '5G:', '4G:', '3G:', 'Operacinė sistema:', 'Baterijos tipas:', 'Pagrindinė kamera:', 'Priekinė kamera:', 'WiFi:', 'GPS:', 'Procesoriaus tipas:', 'Bluetooth:', 'Operatyvinė atmintis (RAM):', 'Atminties kortelės tipas:', 'Skirta vaikams:', 'Svoris:', 'Spalva:', 'SIM:', 'Kita informacija:', 'USB jungtys:', 'Baterijos talpa (mAh):', 'Aukštis:', 'Gylis:', 'Plotis:', 'Pardavėjas:', 'Ekrano įstrižainė:', 'Rūšis:', 'Vidinė atmintis:', 'Maksimali raiška:', 'Visos šio ženklo prekės:']
36 ['67919111', 'Planšetiniai kompiuteriai', 'RT3-BK/OL', '1 vnt.', '0,25 x 0,15 x 0,07 m, 0,7 kg', 'OukiTel', 'IPS', 'Ne', 'Taip', 'Taip', 'Android 12', 'Lithium-Ion (Li-Ion)', '16 MP', '8 MP', 'Yra', 'Yra', 'Octa-core', 'Yra', '4000 MB', 'Micro SD', 'Ne', '0.5381 kg', 'Juoda', '

In [ ]:
# createMainTable('VarlePigu.db')

True

In [9]:
# Plansete Pigu scraper

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
# driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
driver = webdriver.Chrome(options=opcijos)
url = 'https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets?page=18'
driver.get(url)
time.sleep(5)
source = driver.page_source

bs = BeautifulSoup(source, 'html.parser')
max_page_number = bs.find('a', {'class':'pageSlot-7'}).text
last_page = int(max_page_number)

counts = 1001
psl = 18
# iki_kokio_psl = 18
# while psl < iki_kokio_psl:
while psl < last_page + 1:
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    p_nuorodos = []
    linkai = bs.find_all('div', {'class':'c-product-card__title'})
    for linkas in linkai:
        link = linkas.find('a')['href']
        # print(link)
        p_nuorodos.append(link)
        
    for nuoroda in p_nuorodos:
        print(psl, nuoroda)
        url = nuoroda
        driver.get(url)
        # time.sleep(random.randint(5,10))
        time.sleep(3)
        source = driver.page_source
        
        cols_i_have = getColNamesList(table='PlansetePigu', dbname='VarlePigu.db')
        
        duomenys = {}
        for col in cols_i_have:
            duomenys[col] = None
            
        duomenys['nuoroda'] = nuoroda
        
        bs = BeautifulSoup(source, 'html.parser')
        firstInfo = bs.find('div', {'class':'c-product__info'}).find('h1', {'class':'c-product__name'}).text.strip()
        duomenys['firstInfo'] = firstInfo

        secondInfo = bs.find('div', {'class':'c-accordion-item__title'}).text.replace('Prekės aprašymas:', '').strip()
        duomenys['secondInfo'] = secondInfo

        # kaina = bs.find('div', {'class':'c-price h-price--xx-large h-price--new'}).text.strip()
        # print(kaina)

        kaina = bs.find('div', {'class':'c-product__media'})
        price = eval(kaina['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
        kaina2 = price['dataLayerItem'].get('price', 'nenurodyta')
        duomenys['kaina'] = kaina2

        specifikacija_keys = []
        specifikacija_values = []
        specifikacija = bs.find('div', {'class':'h-row h-pl--8 h-pr--8'}).find_all('tr')
        for trow in specifikacija:
            td = trow.find_all('td')
            key = td[0].text.strip()
            value = td[1].text.strip()
            specifikacija_keys.append(key)
            specifikacija_values.append(value)

        for k, v in zip(specifikacija_keys, specifikacija_values):
            duomenys[k] = v
            
        if kaina2 == 'nenurodyta':
            continue
        else:
            cols_i_need = list(duomenys.keys())
        
            data = tuple(duomenys.values())
            
            cols_i_have = getColNamesList(table='PlansetePigu', dbname='VarlePigu.db')
            
            alterTableAddColumn(cols_i_need, cols_i_have, 'PlansetePigu', 'VarlePigu.db')
            
            writeMany(data, 'PlansetePigu', 'VarlePigu.db')
            print(f'Įrašytas įrašas Nr {counts}, puslapis {psl}')
            counts = counts +1
            
    print(f'Įrašytas puslapis {psl}')
    psl = psl +1
    # if psl < iki_kokio_psl:
    if psl <= last_page:
        next_nuoroda = f'https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets?page={psl}'
        driver.get(next_nuoroda)
        time.sleep(5)
        print(f'kitas puslapis: {psl}')
        
driver.quit()
print('Baigta!!!')


18 https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-ipad-air-13-m2-wi-fi-512gb?id=84184456
Įrašytas įrašas Nr 1001, puslapis 18
18 https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-ipad-pro-13-m4-wi-fi-cellular?id=84198415
Įrašytas įrašas Nr 1002, puslapis 18
18 https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-ipad-pro-13-m4-wi-fi-2tb?id=84201865
Įrašytas įrašas Nr 1003, puslapis 18
18 https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-ipad-air-11-m2-wi-fi-cellular?id=84183376
Įrašytas įrašas Nr 1004, puslapis 18
18 https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-ipad-air-11-m2-wi-fi-cellular?id=84183892
Įrašytas įrašas Nr 1005, puslapis 18
18 https://pigu.lt/lt/plansetiniai-kompiuteriai/plansetes-tablets/plansetinis-kompiuteris-ipad-air-11-m2-wi-fi-cellular?id=84184008
Įrašytas įrašas Nr 1006, puslap